You are given an m x n grid grid where:

'.' is an empty cell.
'#' is a wall.
'@' is the starting point.
Lowercase letters represent keys.
Uppercase letters represent locks.
You start at the starting point and one move consists of walking one space in one of the four cardinal directions. You cannot walk outside the grid, or walk into a wall.

If you walk over a key, you can pick it up and you cannot walk over a lock unless you have its corresponding key.

For some 1 <= k <= 6, there is exactly one lowercase and one uppercase letter of the first k letters of the English alphabet in the grid. This means that there is exactly one key for each lock, and one lock for each key; and also that the letters used to represent the keys and locks were chosen in the same order as the English alphabet.

Return the lowest number of moves to acquire all keys. If it is impossible, return -1.

 

Example 1:


Input: grid = ["@.a..","###.#","b.A.B"]
Output: 8
Explanation: Note that the goal is to obtain all the keys not to open all the locks.
Example 2:


Input: grid = ["@..aA","..B#.","....b"]
Output: 6
Example 3:


Input: grid = ["@Aa"]
Output: -1
 

Constraints:

m == grid.length
n == grid[i].length
1 <= m, n <= 30
grid[i][j] is either an English letter, '.', '#', or '@'. 
There is exactly one '@' in the grid.
The number of keys in the grid is in the range [1, 6].
Each key in the grid is unique.
Each key in the grid has a matching lock.

In [ ]:
class Solution:
    def shortestPathAllKeys(self, grid: list[str]) -> int:
        from collections import deque

        m, n = len(grid), len(grid[0])
        all_keys = 0
        start = (0, 0)

        # Find starting point and total keys
        for i in range(m):
            for j in range(n):
                char = grid[i][j]
                if char == '@':
                    start = (i, j)
                elif 'a' <= char <= 'f':
                    #. we are bit masking all the keys.
                    # For 'a', 1 << 0 → 0b000001
                    # For 'c', 1 << 2 → 0b000100
                    all_keys |= (1 << (ord(char) - ord('a')))

        visited = set()

        def dfs(x, y, keys, steps):
            # If all keys collected
            if keys == all_keys:
                return steps

            if (x, y, keys) in visited:
                return float('inf')
            visited.add((x, y, keys))

            res = float('inf')
            for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    ch = grid[nx][ny]
                    # if its a wall, we cant go
                    if ch == '#':
                        continue
                    # if its a lock we dont have the key for it, we can't go.
                    if 'A' <= ch <= 'F' and not (keys & (1 << (ord(ch.lower()) - ord('a')))):
                        continue
                    new_keys = keys
                    if 'a' <= ch <= 'f':
                        new_keys |= (1 << (ord(ch) - ord('a')))
                    
                    res = min(res, dfs(nx, ny, new_keys, steps + 1))
            visited.remove((x, y, keys))
            return res

        ans = dfs(start[0], start[1], 0, 0)
        return ans if ans != float('inf') else -1

# tc - Up to O(4^L * 2^K) — where L is the number of cells and K is number of keys (max 6 → 64 states).

# appraoch 2:
| Criteria                     | ✅ Met? | Reason                                                    |
| ---------------------------- | ------ | --------------------------------------------------------- |
| **Shortest path guaranteed** | ✅      | BFS gives shortest path in an unweighted graph            |
| **Efficient key tracking**   | ✅      | Bitmask handles keys in constant-size space (≤ 64 states) |
| **Avoids recomputation**     | ✅      | Uses `(i, j, keys)` as state in `visited`                 |
| **Minimal complexity**       | ✅      | `O(m × n × 2^k)` — best possible for this type of problem |


In [ ]:
from collections import deque

class Solution:
    def shortestPathAllKeys(self, grid: list[str]) -> int:
        m, n = len(grid), len(grid[0])
        all_keys = 0
        start = None

        # Find start position and all keys
        for i in range(m):
            for j in range(n):
                cell = grid[i][j]
                if cell == '@':
                    start = (i, j)
                elif 'a' <= cell <= 'f':
                    all_keys |= (1 << (ord(cell) - ord('a')))

        # BFS queue: (x, y, collected_keys_bitmask)
        queue = deque()
        visited = set()

        queue.append((start[0], start[1], 0, 0))  # (i, j, collected_keys, steps)
        visited.add((start[0], start[1], 0))

        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        while queue:
            i, j, keys, steps = queue.popleft()

            if keys == all_keys:
                return steps  # all keys collected

            for dx, dy in directions:
                ni, nj = i + dx, j + dy
                if 0 <= ni < m and 0 <= nj < n:
                    cell = grid[ni][nj]

                    if cell == '#':
                        continue  # wall

                    new_keys = keys
                    if 'a' <= cell <= 'f':
                        # Collect key
                        new_keys |= (1 << (ord(cell) - ord('a')))

                    if 'A' <= cell <= 'F':
                        # Check if we have the key
                        if not (keys & (1 << (ord(cell.lower()) - ord('a')))):
                            continue

                    if (ni, nj, new_keys) not in visited:
                        visited.add((ni, nj, new_keys))
                        queue.append((ni, nj, new_keys, steps + 1))

        return -1  # can't collect all keys




### ✅ **Time Complexity: O(m × n × 2^k)**

Where:

* `m × n` = number of cells in the grid
* `k` = number of keys (`'a'` to `'f'`, so max `k = 6`)
* `2^k` = all possible combinations of key collections (bitmasking)

**Why this?**

* Each state is represented by: `(i, j, key_mask)`
* There are at most `m × n × 2^k` possible states.
* Each state is visited **at most once**, and from each state we perform **constant operations** (at most 4 directions).

---

### ✅ **Space Complexity: O(m × n × 2^k)**

**Why this?**

* You maintain a `visited[i][j][key_mask]` structure to track visited states.
* You store up to `m × n × 2^k` states in the queue during BFS.

---

### 📌 Example Bound:

* If `m = n = 30` (grid size up to 30×30) and `k = 6` (max keys),
* Then time and space complexity is:
  **O(900 × 64) = O(57,600)** → Efficient.



["@.a.#",
 "###.#",
 "b.A.B"]

- Start: @ at (0,0)
- Keys: a and b
- Locks: A, B
- Goal: Collect all keys (bitmask = 0b11 = 3)


| Step | Queue Front       | Action                                 | New States Added to Queue                | Explanation                      |
| ---- | ----------------- | -------------------------------------- | ---------------------------------------- | -------------------------------- |
| 0    | `(0,0), mask=000` | Move right                             | `(0,1), mask=000`                        | `.` is empty, add to queue       |
|      |                   | Move down → wall (`#`)                 | -                                        | Wall, skip                       |
| 1    | `(0,1), mask=000` | Move right to `(0,2)` → key `a`        | `(0,2), mask=001`                        | Found `a`, update bitmask        |
|      |                   | Move left → already visited            | -                                        | Already seen `(0,0), mask=000`   |
| 2    | `(0,2), mask=001` | Move right → wall                      | -                                        | Wall, skip                       |
|      |                   | Move down to `(1,2)` → wall            | -                                        | Wall, skip                       |
|      |                   | Move left to `(0,1)`                   | `(0,1), mask=001`                        | Revisit with new key mask        |
| 3    | `(0,1), mask=001` | Move down to `(1,1)` → wall            | -                                        | Wall, skip                       |
|      |                   | Move left to `(0,0)`                   | `(0,0), mask=001`                        | Revisit start with key `a`       |
| 4    | `(0,0), mask=001` | Move down to `(1,0)` → wall            | -                                        | Wall, skip                       |
|      |                   | Move right to `(0,1)`                  | -                                        | Already visited                  |
| 5    | - No progress yet | Now try left side, backtrack & go down | `(2,0), mask=001` (from `(1,0)` down)    | Found `b`, bitmask becomes `011` |
| 6    | `(2,0), mask=011` | Check goal condition                   | ✅ YES, mask == `011`, all keys collected | **Return steps taken so far**    |


